### **yolov1论文**

In [2]:
import torch
from torch import nn
from torchinfo import summary

**一、搭建模型**

**backbone采用VGG架构**

In [3]:
def get_yolov1(num_classes = 20):
    """获取yolov1o模型"""
    return nn.Sequential(
        nn.Conv2d(3,64,kernel_size = 7,stride = 2,padding = 3),nn.LeakyReLU(),
        nn.MaxPool2d(2,2),                    # k = 2,s = 2的MaxPool2d层使图像分辨率减半
        nn.Conv2d(64,192,kernel_size = 3,padding = 1),nn.LeakyReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(192,128,1),nn.LeakyReLU(),
        nn.Conv2d(128,256,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(256,256,1),nn.LeakyReLU(),
        nn.Conv2d(256,512,3,padding = 1),nn.LeakyReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(512,256,1),nn.LeakyReLU(),
        nn.Conv2d(256,512,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(512,256,1),nn.LeakyReLU(),
        nn.Conv2d(256,512,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(512,256,1),nn.LeakyReLU(),
        nn.Conv2d(256,512,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(512,256,1),nn.LeakyReLU(),
        nn.Conv2d(256,512,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(512,512,1),nn.LeakyReLU(),
        nn.Conv2d(512,1024,3,padding = 1),nn.LeakyReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(1024,512,1),nn.LeakyReLU(),
        nn.Conv2d(512,1024,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(1024,512,1),nn.LeakyReLU(),
        nn.Conv2d(512,1024,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(1024,1024,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(1024,1024,3,stride = 2,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(1024,1024,3,padding = 1),nn.LeakyReLU(),
        nn.Conv2d(1024,1024,3,padding = 1),nn.LeakyReLU(),
        nn.Flatten(),nn.Linear(7 * 7 * 1024,4096),
        nn.Linear(4096,7 * 7 * 30)
    )

In [4]:
net = get_yolov1()

In [5]:
input = torch.rand((1,3,448,448)) # 通道优先
y_pred = net(input).reshape(1,30,7,7)
# y_pred = net(input)
print(y_pred.shape)

torch.Size([1, 30, 7, 7])


In [6]:
print(summary(net,input_data = input))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 1470]                 --
├─Conv2d: 1-1                            [1, 64, 224, 224]         9,472
├─LeakyReLU: 1-2                         [1, 64, 224, 224]         --
├─MaxPool2d: 1-3                         [1, 64, 112, 112]         --
├─Conv2d: 1-4                            [1, 192, 112, 112]        110,784
├─LeakyReLU: 1-5                         [1, 192, 112, 112]        --
├─MaxPool2d: 1-6                         [1, 192, 56, 56]          --
├─Conv2d: 1-7                            [1, 128, 56, 56]          24,704
├─LeakyReLU: 1-8                         [1, 128, 56, 56]          --
├─Conv2d: 1-9                            [1, 256, 56, 56]          295,168
├─LeakyReLU: 1-10                        [1, 256, 56, 56]          --
├─Conv2d: 1-11                           [1, 256, 56, 56]          65,792
├─LeakyReLU: 1-12                        [1, 256, 56, 56]       

**二、定义损失函数**

In [ ]:
class LossYolo():
    """yolov1的损失函数"""
    def __init__(self):
        self.coord = 5
        self.noobj = 0.5
